In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = "../Database/database1/"
usdataset = "us_equities_news_dataset.csv"

In [3]:
totalpath = path + usdataset
df = pd.read_csv(totalpath)


In [7]:
# df['release_date'] make to datetime
df['release_date'] = pd.to_datetime(df['release_date'])

In [ ]:
df['weekday'] = df['release_date'].dt.dayofweek
df['weekday'].value_counts().plot(kind='bar')

Dataset:
- filter duplicates and missing values
- only tickers nasdaq100 as of Feb 2020
- all days
- dates: only between 2016-04-17 and 2019-04-16
- columns: title, ticker, release date, content


In [10]:
#put df in chronological order according to release date
df = df.sort_values(by=['release_date'])


# Delete content duplicates

In [12]:
#remove the row element where there is missing values in the content column
df = df.dropna(subset=['content'])

In [13]:
#print number of titles that are more than once in df
print("Number of titles that are more than once in df: ", len(df[df.duplicated(subset=['title'])]))

Number of titles that are more than once in df:  6066


In [14]:
#print number of content that are more than once in df
print("Number of content that are more than once in df: ", len(df[df.duplicated(subset=['content'])]))


Number of content that are more than once in df:  509


In [15]:
#keep only the first occurence of each content
df = df.drop_duplicates(subset=['content'], keep='first')
df.shape

(220996, 10)

# Keep only columns title, content, ticker and release date

In [16]:
#Keep only columns title, content, ticker and release date
df = df[['title', 'content', 'release_date','ticker']]
df.shape


(220996, 4)

# Tickers

In [ ]:
#most recurring tickers
print("Most recurring tickers: ", df['ticker'].value_counts().head(10))

In [ ]:
#the count of elements over the years
print("Count of elements over the years: ", df['release_date'].dt.year.value_counts().sort_index())

In [19]:
#OFFICIAL LIST OF NASDAQ 100 TICKERS (source: official website - in 2020)
nasdaq100_tickers = ['AAPL', 'ADBE', 'ADI', 'ADSK', 'ALGN', 'ALXN', 'AMAT', 'AMGN', 'AMZN', 'ANSS', 'ASML', 'ATVI', 'ADBE', 'BKNG', 'BIDU', 'BIIB', 'BMRN', 'CDNS', 'CERN', 'CHKP', 'CHTR', 'CMCSA', 'COST', 'CSCO', 'CSX', 'CTAS', 'CTSH', 'DLTR', 'EA', 'EBAY', 'EXPE', 'FAST', 'FB', 'FISV', 'FOXA', 'GILD', 'GOOG', 'GOOGL', 'HAS', 'HSIC', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'JD', 'KDP', 'KHC', 'KLAC', 'LBTYA', 'LBTYK', 'LRCX', 'LULU', 'MAR', 'MCHP', 'MDLZ', 'MELI', 'MNST', 'MSFT', 'MU', 'MXIM', 'MYL', 'NFLX', 'NTES', 'NVDA', 'NXPI', 'ORLY', 'PAYX', 'PCAR', 'PEP', 'PYPL', 'QCOM', 'REGN', 'ROST', 'SBUX', 'SIRI', 'SNPS', 'SPLK', 'SWKS', 'SYMC', 'TMUS', 'TSLA', 'TXN', 'ULTA', 'VRSK', 'VRSN', 'VRTX', 'WBA', 'WDAY', 'WDC', 'XEL', 'XLNX', 'ZM']

In [21]:
# print the ticker in df that are in nasdaq100_tickers
df= df[df['ticker'].isin(nasdaq100_tickers)]
df.shape

(67741, 4)

# Release date from 2016-04-17 and 2019-04-16

In [26]:
df = df[(df['release_date'] > '2016-04-17') & (df['release_date'] < '2019-04-16')]
df.shape

(41997, 4)

In [25]:
df = df.reset_index(drop=True)

## weekends

In [27]:
#number of elements in the weekends
print("Number of elements in the weekends: ", len(df[df['release_date'].dt.weekday > 4]))

Number of elements in the weekends:  3803


# Conclusion

52k articles to predict 3 years of weekdays

In [ ]:
#print the average number of elements per date in the weekdays
print("Average number of elements per date in the weekdays: ", len(df[df['release_date'].dt.weekday < 5])/len(df[df['release_date'].dt.weekday < 5]['release_date'].dt.date.unique()))

# Each holidays/weekend to next date

## Dates

In [40]:
path2 = "../Database/price.csv"
df2 = pd.read_csv(path2)

In [42]:
df2['Date'] = pd.to_datetime(df2['Date'])

In [43]:
dates1 = df2['Date'].dt.date.unique()
len(dates1)

755

In [44]:
releasedates = df['release_date'].dt.date.unique()
len(releasedates)


1070

In [ ]:
#the dates that in releasedates but not in dates and its length
diff =  set(releasedates) - set(dates1)
print("Dates that in releasedates but not in dates: ",diff)
print("Length of dates that in releasedates but not in dates: ", len(diff))


Change the dates in df to the next date chronoligically that is in dates

In [ ]:
#assign each date in df that is not in variable dates1 to the first following day that is in the dates1
for i in range(len(df)):
    if df['release_date'].iloc[i].date() not in dates1:
        df['release_date'].iloc[i] = df['release_date'].iloc[i] + pd.Timedelta(days=1)
        while df['release_date'].iloc[i].date() not in dates1:
            df['release_date'].iloc[i] = df['release_date'].iloc[i] + pd.Timedelta(days=1)
            


# Removing too long content articles avoid to do it in next code snippet

In [ ]:
#only keep first 512 TOKENS in content
df['content'] = df['content'].str[:512]


In [ ]:
#show per year the number of elements
df['release_date'].dt.year.value_counts().sort_index().plot(kind='bar')


# Save the database

In [50]:
df

,title,content,release_date,ticker
0,Stocks Poised For More Upside,In my experience there is only one motivatio...,2016-04-18,TSLA
1,U S futures point to lower open amid dropping...,Investing com Wall Street futures pointed to...,2016-04-18,NFLX
2,Netflix shares plunge as subscriber forecasts ...,By Anya George Tharakan and Lisa Richwine Reu...,2016-04-18,NFLX
3,Dow reclaims 18 000 as quarterly scorecards st...,By Noel Randewich Reuters The Dow Jones in...,2016-04-18,NFLX
4,Netherlands stocks higher at close of trade A...,Investing com Netherlands stocks were higher...,2016-04-18,ASML
...,...,...,...,...
41992,Netherlands stocks higher at close of trade A...,Investing com Netherlands stocks were higher...,2019-04-15,ASML
41993,Analysts Estimate Hasbro HAS To Report A Dec...,Wall Street expects a year over year decline i...,2019-04-15,HAS
41994,Apple allies seek billions in U S trial test...,By Stephen Nellis Reuters Apple Inc NASDA...,2019-04-15,INTC
41995,Sweden s Veoneer bets driverless car delay wil...,By Johannes Hellstrom and Esha Vaish STOCKHOLM...,2019-04-15,INTC


In [53]:
df.to_csv("news_df.csv", index=False)